In [0]:
# TRANSFORMACIONES
    # mapValues(f): Realiza operaciones sobre los valores de los pares (K,V)
    # reduceByKey(f): Al llamarlo sobre un RDD de pares clave-valor (K, V),  devuelve otro de pares (K, V) donde los valores de cada  clave se han agregado usando la función dada.
    # groupByKey(f): Al llamarlo sobre un RDD de pares clave-valor (K, V),  devuelve otro de pares (K, seq[V]) donde los valores de  cada clave se han convertido a una secuencia.
    # sortByKey(): Ordena un RDD de pares clave-valor (K, V) por clave.
    # join(rdd): Hace un join de dos rdd de pares (K, V1) y (K,V2) y devuelve otro RDD con claves (K, (V1, V2))

In [0]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_long = palabras.map(lambda elem: (elem, len(elem)))

pal_long.collect()

Out[3]: [('HOLA', 4), ('Que', 3), ('TAL', 3), ('Bien', 4)]

In [0]:
# Realiza una operación (la función argumento dada) sobre los valores de los pares. RDD nuevo.

frutas = sc.parallelize([('P', 'peras, piñas'), ('M', 'manzanas, melones, melocotones'), ('N','naranjas, nectarinas'), ('L', 'limones')])

frutas_MAY = frutas.mapValues(lambda elemento: elemento.upper())

frutas_MAY.collect()

Out[5]: [('P', 'PERAS,PIÑAS'),
 ('M', 'MANZANAS,MELONES,MELOCOTONES'),
 ('N', 'NARANJAS,NECTARINAS'),
 ('L', 'LIMONES')]

In [0]:
# Los valores de cada clave del RDD se han agregado usando la función dada hasta obtener un único valor por clave

r = sc.parallelize([("A", 1), ("C", 4), ("A", 1), ("B", 1), ("B", 4)])

rr = r.reduceByKey(lambda v1, v2: v1 + v2)

print(rr.collect())

[('B', 5), ('C', 4), ('A', 2)]


In [0]:
# Además transforma los 'valores' de las claves 'A' y 'B' a str // 'C' es único, no se suma, por tanto no se aplica la función
# Inconsistencia de tipo, no deberíamos hacer esto

r = sc.parallelize([('A', 1), ('C', 4), ('A', 1), ('B', 1), ('B', 4)])

rr1 = r.reduceByKey(lambda v1, v2: str(v1 + v2))

print(rr1.collect())

[('B', '5'), ('C', 4), ('A', '2')]


In [0]:
# Aquí te daría error, porque hace str(1 + 4) > str('5' + 7) ---> 'TypeError: can only concatenate str (not "int") to str'

r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4), ('B',7)])

rr1 = r.reduceByKey(lambda v1,v2: str(v1+v2))

#print(rr1.collect())

In [0]:
# Agrupa todos los elementos del RDD para obtener un único valor por clave con valor igual a la secuencia de valores
# Es necesario utilizar mapValues(...) para ver el resultado (se suele usar como paso intermedio en un proceso)

r = sc.parallelize([("A", 1), ("C", 2), ("A", 3), ("B", 4), ("B", 5), ("A", 3)])

r2 = r.groupByKey().mapValues(list)    # Concatena los values de igual key
print(r2.collect())

r3 = r.groupByKey().mapValues(len)     # Suma las keys independientemente del value
print(r3.collect())

# Se usa en procesos intermedios. Si lo que queremos una vez agrupados es agregar los datos  (resultado final) ->  “reduceByKey()”

[('B', [4, 5]), ('C', [2]), ('A', [1, 3, 3])]
[('B', 2), ('C', 1), ('A', 3)]


In [0]:
# Ordena por clave un RDD de pares (K,V)

rdd = sc.parallelize([("A", 1), ("B", 6), ("C", 3), ("A", 4), ("A", 5), ("B", 2)])

res = rdd.sortByKey()  # (False): orden inverso

print(res.collect())

[('A', 1), ('A', 4), ('A', 5), ('B', 6), ('B', 2), ('C', 3)]


In [0]:
# Combina dos rdd de pares (K, V1) y (K, V2) basándose en la clave común y devuelve otro RDD con claves (K, (V1, V2))
# En caso de correspondencia de clave en ambos RDDs devuelve el producto cartesiano (formado por todos los pares ordenados que se pueden formar a partir de dos ciertos conjuntos)

rdd1 = sc.parallelize([("A", 1), ("B", 2), ("C", 2)])
rdd2 = sc.parallelize([("A", 4), ("B", 5), ("A", 7)])

rddjoin = rdd1.join(rdd2)

rddjoin.collect()

Out[8]: [('B', (2, 5)), ('A', (1, 4)), ('A', (1, 7))]

In [0]:
# Une dos RDD basados en una clave común y devuelve una nueva RDD que contiene todas las tuplas de la RDD izquierda (primera) y las tuplas coincidentes de la RDD derecha (segunda, entre paréntesis). 
    # Si hay un valor en la RDD izquierda que no está en la RDD derecha, se asigna un valor NULO. 
    # Si un valor esta en la RDD derecha solo, no aparecerá en el join.

rdd_left = sc.parallelize([("A", 1), ("B", 2), ("C", 3)])
rdd_right = sc.parallelize([("A", 4), ("A", 5), ("B", 6), ("D", 7)])  # no 'C'

rdd_left_join = rdd_left.leftOuterJoin(rdd_right)

rdd_left_join.collect()

Out[9]: [('B', (2, 6)), ('C', (3, None)), ('A', (1, 4)), ('A', (1, 5))]

In [0]:
# Une dos RDD basados en una clave común y devuelve una nueva RDD que contiene todas las tuplas de la RDD derecha (segunda, entre paréntesis) y las tuplas coincidentes de la RDD izquierda (primera). 
    # Si hay un valor en la RDD derecha que no está en la RDD izquierda, se asigna un valor NULO. 
    # Si un valor esta en la RDD izquierda solo, no aparecerá en el join.

rdd_left = sc.parallelize([("A", 1), ("B", 2), ("C", 3)])             # no 'D'
rdd_right = sc.parallelize([("A", 4), ("A", 5), ("B", 6), ("D", 7)])

rdd_right_join = rdd_left.rightOuterJoin(rdd_right)    # o hacer un leftjoin intercambiando las rdd (solo varía ordenamiento resultado)

rdd_right_join.collect()

Out[12]: [('B', (2, 6)), ('D', (None, 7)), ('A', (1, 4)), ('A', (1, 5))]

In [0]:
# Salen todos los resultados, como si usaramos right y left a la vez

rdd_left = sc.parallelize([("A", 1), ("B", 2), ("C", 3)])
rdd_right = sc.parallelize([("A", 4), ("A", 5), ("B", 6), ("D", 7)])

full_outer_join = rdd_left.fullOuterJoin(rdd_right)   # el 'rdd' que va primero solo influye en el ordenamiento del resultado.

full_outer_join.collect()

Out[15]: [('B', (2, 6)),
 ('D', (None, 7)),
 ('C', (3, None)),
 ('A', (1, 4)),
 ('A', (1, 5))]